<a href="https://colab.research.google.com/github/VishalShah1999/Analysis_of_Meteorological_Data/blob/master/AiStock_EMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 6.4 MB 15.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!pip install pandas_ta

     |████████████████████████████████| 115 kB 7.8 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=f3c3bdf87308d86f2b021b90d5622b3cc435caf4ca09e75994e188a30b20afc4
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [ ]:
if __name__ == '__main__':
    
    # Getting the data
    nvda = yf.Ticker('NVDA')
    df = nvda.history(period='1y')[['Open', 'High', 'Low', 'Close', 'Volume']]
    
    # Create a pandas_ta strategy
    moving_averages = ta.Strategy(
        name="EMA_8_13_21",
        ta=[
            {"kind": "ema", "length": 8},
            {"kind": "ema", "length": 13},
            {"kind": "ema", "length": 21}
        ]
    )
    
    # Add bulk indicators
    df.ta.strategy(moving_averages, append=True)

In [3]:
import pandas_ta as ta
import yfinance as yf
import plotly.graph_objects as go

# Fetch the data
df = yf.Ticker('AMD').history(period='6mo')[['Open', 'Close', 'High', 'Low', 'Volume']]

# Add the indicators
moving_averages = ta.Strategy(
    name="moving indicators",
    ta=[
        {"kind": "ema", "length": 8},
        {"kind": "ema", "length": 13},
        {"kind": "ema", "length": 21},
        {"kind": "ema", "length": 11},
        {"kind": "ema", "length": 31}
    ]
)

# Creating new columns for every EMA's
df.ta.strategy(moving_averages)

# Creating CandleStick Chart
fig = go.Figure(data=[
    go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        increasing_line_color='green',
        decreasing_line_color='red',
        showlegend=False,
    ),
])

# Make it pretty
layout = go.Layout(
    plot_bgcolor='#efefef',
    # Font Families
    font_family='Monospace',
    font_color='#000000',
    font_size=20,
    xaxis=dict(
        rangeslider=dict(
            visible=False
        ))
)

# Adding Trace for every individual EMA's
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['EMA_8'],
        line=dict(color='#000', width=2),
        name='EMA_8'
    )
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['EMA_13'],
        line=dict(color='green', width=2),
        name='EMA_13'
    )
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['EMA_21'],
        line=dict(color='red', width=2),
        name='EMA_21'
    )
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['EMA_11'],
        line=dict(color='blue', width=2),
        name='EMA_11'
    )
)

fig.add_trace(
    go.Scatter(
        x=df.index,
        y=df['EMA_31'],
        line=dict(color='cyan', width=2),
        name='EMA_31'
    )
)
fig.update_layout(layout)
# Display (in browser by default)
fig.show()

In [4]:
# Counting Ratios
print("************************************************************************************************************************************************")
print("Counting Ratios")
buy, sell = 0, 0
def ratio_13(x,y):
  buy, sell = 0, 0
  for i in range(-1,-14,-1):
    if x[i] > y[i]:
      buy += 1
    else:
      sell += 1
  return buy, sell

def ratio_21(x,y):
  buy, sell = 0, 0
  for i in range(-1,-22,-1):
    if x[i] > y[i]:
      buy += 1
    else:
      sell += 1
  return buy, sell

def ratio_11(x,y):
  buy, sell = 0, 0
  for i in range(-1,-12,-1):
    if x[i] > y[i]:
      buy += 1
    else:
      sell += 1
  return buy, sell

def ratio_31(x,y):
  buy, sell = 0, 0
  for i in range(-1,-32,-1):
    if x[i] > y[i]:
      buy += 1
    else:
      sell += 1
  return buy, sell

************************************************************************************************************************************************
Counting Ratios


In [11]:
red_count = 0
def counting_candles_13(df):
  red_count, daysOlderRed, daysOlderGreen, vol_red, vol_green = 0, 0, 0, 0, 0
  biggest_red, biggest_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG = 0, 0, 0, 0, 0, 0, 0, 0
  open = df['Open'][-13:]
  close = df['Close'][-13:]
  volume = df['Volume'][-13:]
  for i in range(len(open)):
    if open[i] > close[i]:
      red_count += 1
      if i == 0:
        biggest_red = (close[i] - open[i])/close[i]
        daysOlderRed = 13 - i
        vol_red = volume[i]
        prev_closeR = close[i]
        cur_closeR = close[i]
        cur_openR = open[i]
      elif biggest_red < ((close[i] - open[i])/close[i-1]):
        biggest_red = ((close[i] - open[i])/close[i-1])
        daysOlderRed = 13 - i
        vol_red = volume[i]
        prev_closeR = close[i-1]
        cur_closeR = close[i]
        cur_openR = open[i]
    elif open[i] < close[i]:
      if i == 0:
        biggest_green = (close[i] - open[i])/close[i]
        daysOlderGreen = 13 - i
        vol_green = volume[i]
        prev_closeG = close[i]
        cur_closeG = close[i]
        cur_openG = open[i]
      elif biggest_green < ((close[i]-open[i])/close[i-1]):
        biggest_green = ((close[i]-open[i])/close[i-1])
        daysOlderGreen = 13 - i
        vol_green = volume[i]
        prev_closeG = close[i-1]
        cur_closeG = close[i]
        cur_openG = open[i]
  return red_count, 13-red_count, biggest_red, biggest_green, daysOlderRed, daysOlderGreen, vol_red, vol_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG

def counting_candles_21(df):
  red_count, daysOlderRed, daysOlderGreen, vol_red, vol_green = 0, 0, 0, 0, 0
  biggest_red, biggest_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG = 0, 0, 0, 0, 0, 0, 0, 0
  open = df['Open'][-21:]
  close = df['Close'][-21:]
  volume = df['Volume'][-21:]
  for i in range(len(open)):
    if open[i] > close[i]:
      red_count += 1
      if i == 0:
        biggest_red = (close[i] - open[i])/close[i]
        daysOlderRed = 21 - i
        vol_red = volume[i]
        prev_closeR = close[i]
        cur_closeR = close[i]
        cur_openR = open[i]
      elif biggest_red < ((close[i]-open[i])/close[i-1]):
        biggest_red = ((close[i]-open[i])/close[i-1])
        daysOlderRed = 21 - i
        vol_red = volume[i]
        prev_closeR = close[i-1]
        cur_closeR = close[i]
        cur_openR = open[i]
    elif open[i] < close[i]:
      if i == 0:
        biggest_green = (close[i] - open[i])/close[i]
        daysOlderGreen = 21 - i
        vol_green = volume[i]
        prev_closeG = close[i]
        cur_closeG = close[i]
        cur_openG = open[i]
      elif biggest_green < ((close[i]-open[i])/close[i-1]):
        biggest_green = ((close[i]-open[i])/close[i-1])
        daysOlderGreen = 21 - i
        vol_green = volume[i]
        prev_closeG = close[i-1]
        cur_closeG = close[i]
        cur_openG = open[i]
  return red_count, 21-red_count, biggest_red, biggest_green, daysOlderRed, daysOlderGreen, vol_red, vol_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG

def counting_candles_11(df):
  red_count, daysOlderRed, daysOlderGreen, vol_red, vol_green = 0, 0, 0, 0, 0
  biggest_red, biggest_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG = 0, 0, 0, 0, 0, 0, 0, 0
  open = df['Open'][-11:]
  close = df['Close'][-11:]
  volume = df['Volume'][-11:]
  for i in range(len(open)):
    if open[i] > close[i]:
      red_count += 1
      if i == 0:
        biggest_red = (close[i] - open[i])/close[i]
        daysOlderRed = 11 - i
        vol_red = volume[i]
        prev_closeR = close[i]
        cur_closeR = close[i]
        cur_openR = open[i]
      elif biggest_red < ((close[i]-open[i])/close[i-1]):
        biggest_red = ((close[i]-open[i])/close[i-1])
        daysOlderRed = 11 - i
        vol_red = volume[i]
        prev_closeR = close[i-1]
        cur_closeR = close[i]
        cur_openR = open[i]
    elif open[i] < close[i]:
      if i == 0:
        biggest_green = (close[i] - open[i])/close[i]
        daysOlderGreen = 11 - i
        vol_green = volume[i]
        prev_closeG = close[i]
        cur_closeG = close[i]
        cur_openG = open[i]
      elif biggest_green < ((close[i]-open[i])/close[i-1]):
        biggest_green = ((close[i]-open[i])/close[i-1])
        daysOlderGreen = 11 - i
        vol_green = volume[i]
        prev_closeG = close[i-1]
        cur_closeG = close[i]
        cur_openG = open[i]
  return red_count, 11-red_count, biggest_red, biggest_green, daysOlderRed, daysOlderGreen, vol_red, vol_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG

def counting_candles_31(df):
  red_count, daysOlderRed, daysOlderGreen, vol_red, vol_green = 0, 0, 0, 0, 0
  biggest_red, biggest_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG = 0, 0, 0, 0, 0, 0, 0, 0
  open = df['Open'][-31:]
  close = df['Close'][-31:]
  volume = df['Volume'][-31:]
  for i in range(len(open)):
    if open[i] > close[i]:
      red_count += 1
      if i == 0:
        biggest_red = (close[i] - open[i])/close[i]
        daysOlderRed = 31 - i
        vol_red = volume[i]
        prev_closeR = close[i]
        cur_closeR = close[i]
        cur_openR = open[i]
      elif biggest_red < ((close[i]-open[i])/close[i-1]):
        biggest_red = ((close[i]-open[i])/close[i-1])
        daysOlderRed = 31 - i
        vol_red = volume[i]
        prev_closeR = close[i-1]
        cur_closeR = close[i]
        cur_openR = open[i]
    elif open[i] < close[i]:
      if i == 0:
        biggest_green = (close[i] - open[i])/close[i]
        daysOlderGreen = 31 - i
        vol_green = volume[i]
        prev_closeG = close[i]
        cur_closeG = close[i]
        cur_openG = open[i]
      elif biggest_green < ((close[i]-open[i])/close[i-1]):
        biggest_green = ((close[i]-open[i])/close[i-1])
        daysOlderGreen = 31 - i
        vol_green = volume[i]
        prev_closeG = close[i-1]
        cur_closeG = close[i]
        cur_openG = open[i]
  return red_count, 31-red_count, biggest_red, biggest_green, daysOlderRed, daysOlderGreen, vol_red, vol_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG

In [6]:
def total_avg_13(df):
  close = df['Close'][-13:]
  return (close[-1] - close[0])/close[0]

def total_avg_21(df):
  close = df['Close'][-21:]
  return (close[-1] - close[0])/close[0]

def total_avg_11(df):
  close = df['Close'][-11:]
  return (close[-1] - close[0])/close[0]

def total_avg_31(df):
  close = df['Close'][-31:]
  return (close[-1] - close[0])/close[0]

In [23]:
d = dict()
buy, sell = ratio_13(df['EMA_8'],df['EMA_13'])
red, green, big_red, big_green, older_red, older_green, vol_red, vol_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG = counting_candles_13(df)
total_avg = total_avg_13(df)
d['EMA_13'] = {'AMD': {'crossChange': {'crossDown': sell, 'crossUp': buy}, 'RedGreenCandles': {'Red': red, 'Green': green}, 'UpDownCandles': {'Biggest Red': {'Value': big_red, 'Previous Close': prev_closeR, 'Current Close': cur_closeR, 'Current Open': cur_openR}, 'Biggest Green': {'Value': big_green, 'Previous Close': prev_closeG, 'Current Close': cur_closeG, 'Current Open': cur_openG}}, 'DaysOlder': {'Red': older_red, 'Green':older_green}, 'Volume': {'Red': vol_red, 'Green': vol_green}, 'Total Average': total_avg}}

buy, sell = ratio_21(df['EMA_8'], df['EMA_21'])
red, green, big_red, big_green, older_red, older_green, vol_red, vol_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG = counting_candles_21(df)
total_avg = total_avg_21(df)
d['EMA_21'] = {'AMD': {'crossChange': {'crossDown': sell, 'crossUp': buy}, 'RedGreenCandles': {'Red': red, 'Green': green}, 'UpDownCandles': {'Biggest Red': {'Value': big_red, 'Previous Close': prev_closeR, 'Current Close': cur_closeR, 'Current Open': cur_openR}, 'Biggest Green': {'Value': big_green, 'Previous Close': prev_closeG, 'Current Close': cur_closeG, 'Current Open': cur_openG}}, 'DaysOlder': {'Red': older_red, 'Green':older_green}, 'Volume': {'Red': vol_red, 'Green': vol_green}, 'Total Average': total_avg}}

buy, sell = ratio_11(df['EMA_8'], df['EMA_11'])
red, green, big_red, big_green, older_red, older_green, vol_red, vol_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG = counting_candles_11(df)
total_avg = total_avg_11(df)
d['EMA_11'] = {'AMD': {'crossChange': {'crossDown': sell, 'crossUp': buy}, 'RedGreenCandles': {'Red': red, 'Green': green}, 'UpDownCandles': {'Biggest Red': {'Value': big_red, 'Previous Close': prev_closeR, 'Current Close': cur_closeR, 'Current Open': cur_openR}, 'Biggest Green': {'Value': big_green, 'Previous Close': prev_closeG, 'Current Close': cur_closeG, 'Current Open': cur_openG}}, 'DaysOlder': {'Red': older_red, 'Green':older_green}, 'Volume': {'Red': vol_red, 'Green': vol_green}, 'Total Average': total_avg}}

buy, sell = ratio_31(df['EMA_8'], df['EMA_31'])
red, green, big_red, big_green, older_red, older_green, vol_red, vol_green, prev_closeR, cur_closeR, cur_openR, prev_closeG, cur_closeG, cur_openG = counting_candles_31(df)
total_avg = total_avg_31(df)
d['EMA_31'] = {'AMD': {'crossChange': {'crossDown': sell, 'crossUp': buy}, 'RedGreenCandles': {'Red': red, 'Green': green}, 'UpDownCandles': {'Biggest Red': {'Value': big_red, 'Previous Close': prev_closeR, 'Current Close': cur_closeR, 'Current Open': cur_openR}, 'Biggest Green': {'Value': big_green, 'Previous Close': prev_closeG, 'Current Close': cur_closeG, 'Current Open': cur_openG}}, 'DaysOlder': {'Red': older_red, 'Green':older_green}, 'Volume': {'Red': vol_red, 'Green': vol_green}, 'Total Average': total_avg}}

In [24]:
d['EMA_13']

{'AMD': {'DaysOlder': {'Green': 10, 'Red': 7},
  'RedGreenCandles': {'Green': 7, 'Red': 6},
  'Total Average': 0.02231055226383385,
  'UpDownCandles': {'Biggest Green': {'Current Close': 114.25,
    'Current Open': 107.93000030517578,
    'Previous Close': 105.23999786376953,
    'Value': 0.06005321002576697},
   'Biggest Red': {'Current Close': 120.08000183105469,
    'Current Open': 120.31999969482422,
    'Previous Close': 122.76000213623047,
    'Value': -0.0019550167773962597}},
  'Volume': {'Green': 97206000, 'Red': 103599500},
  'crossChange': {'crossDown': 10, 'crossUp': 3}}}